In [73]:
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import codecs
import json
import itertools
import csv

def read_dataset(path):
  with codecs.open(path, 'r', 'utf-8') as myFile:
    content = myFile.read()
  dataset = json.loads(content)
  return dataset

path = './pizza_request_dataset.json'
dataset = read_dataset(path)

print 'The dataset contains %d samples.' %(len(dataset))
print 'Available attributes: ', sorted(dataset[0].keys())
print 'First post:'
print json.dumps(dataset[0], sort_keys=True, indent=2)

successes = [r['requester_received_pizza'] for r in dataset]
success_rate = 100.0 * sum(successes) / float(len(successes))
print 'The average success rate is: %.2f%%' %(success_rate)

shuffle = np.random.permutation(np.arange(len(dataset)))

The dataset contains 5671 samples.
Available attributes:  [u'giver_username_if_known', u'in_test_set', u'number_of_downvotes_of_request_at_retrieval', u'number_of_upvotes_of_request_at_retrieval', u'post_was_edited', u'request_id', u'request_number_of_comments_at_retrieval', u'request_text', u'request_text_edit_aware', u'request_title', u'requester_account_age_in_days_at_request', u'requester_account_age_in_days_at_retrieval', u'requester_days_since_first_post_on_raop_at_request', u'requester_days_since_first_post_on_raop_at_retrieval', u'requester_number_of_comments_at_request', u'requester_number_of_comments_at_retrieval', u'requester_number_of_comments_in_raop_at_request', u'requester_number_of_comments_in_raop_at_retrieval', u'requester_number_of_posts_at_request', u'requester_number_of_posts_at_retrieval', u'requester_number_of_posts_on_raop_at_request', u'requester_number_of_posts_on_raop_at_retrieval', u'requester_number_of_subreddits_at_request', u'requester_received_pizza', u'

In [69]:
#read train data

train = []
test = []

for i in range(int(len(shuffle) * 0.7)):
    train.append(dataset[shuffle[i]])

for i in range(int(len(shuffle) * 0.3)):
    test.append(dataset[shuffle[len(shuffle) - i - 1]])
    
train_labels = np.zeros(len(train))
test_labels = np.zeros(len(test))

train_data_text = []
test_data_text = []

for i in range(len(train_labels)):
    train_labels[i] = train[i].get('requester_received_pizza')
    train_data_text.append(train[i].get('request_text_edit_aware'))

for i in range(len(test_labels)):
    test_labels[i] = test[i].get('requester_received_pizza')
    test_data_text.append(test[i].get('request_text_edit_aware'))

vec_train = CountVectorizer()
tokenized_train_data = vec_train.fit_transform(train_data_text)

vec_test = CountVectorizer(vocabulary=vec_train.vocabulary_)
tokenized_test_data = vec_test.fit_transform(test_data_text)

print tokenized_train_data.shape
print tokenized_test_data.shape


(3969, 12291)
(1701, 12291)


In [70]:
LR_train = LogisticRegression()
LR_train.fit(tokenized_train_data,train_labels)

test_pred = LR_train.predict(tokenized_test_data)

print(classification_report(test_labels, test_pred))


             precision    recall  f1-score   support

        0.0       0.77      0.86      0.81      1277
        1.0       0.35      0.23      0.28       424

avg / total       0.66      0.70      0.68      1701



In [78]:
#create predictions for Kaggle
test_data = pd.read_json('data/test.json')
test_data_text = test_data.request_text_edit_aware

tokenized_test_data = vec_test.fit_transform(test_data_text)

test_pred = LR_train.predict(tokenized_test_data)

predictions = pd.DataFrame()
predictions['request_id'] = test_data.request_id
predictions['requester_received_pizza'] = test_pred
predictions = predictions.set_index('request_id')

predictions.to_csv('NLP_submission.csv')